In [0]:
%pip install "unstructured[pdf]" layoutparser databricks-sdk pypdf
dbutils.library.restartPython()

In [0]:
import os
import pandas as pd
import uuid
from unstructured.partition.pdf import partition_pdf
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole
from utils import *

In [0]:
# --- ⚙️ CONFIGURACIÓN DINÁMICA ---
dbutils.widgets.text("catalog", "workspace")
dbutils.widgets.text("schema", "default")
dbutils.widgets.text("volume_path", "/Volumes/workspace/default/test1/mydocuments/")
# 🔥 NUEVO PARAMETRO: Para obligar a leer todo aunque ya exista
dbutils.widgets.dropdown("force_refresh", "false", ["true", "false"], "Forzar Reprocesamiento")

CATALOG = dbutils.widgets.get("catalog")
SCHEMA = dbutils.widgets.get("schema")
VOLUME_PATH = dbutils.widgets.get("volume_path")
FORCE_REFRESH = dbutils.widgets.get("force_refresh") == "true"
TABLE_NAME = f"{CATALOG}.{SCHEMA}.pdf_hierarchical_knowledge"

print(f"🔧 Configuración:")
print(f"   📂 Ruta Volumen: {VOLUME_PATH}")
print(f"   💾 Tabla Destino: {TABLE_NAME}")
print(f"   ⚠️ Force Refresh: {FORCE_REFRESH}")

In [0]:
# 1. Obtener archivos en el Volumen (Fuente)
try:
    files = dbutils.fs.ls(VOLUME_PATH)
    # Normalizamos las rutas aquí mismo
    all_pdf_files = [normalizar_ruta(f.path) for f in files if f.name.endswith(".pdf")]
    print(f"📂 Archivos en carpeta ({len(all_pdf_files)}):")
    if all_pdf_files: print(f"   Ejemplo: {all_pdf_files[0]}")
except Exception as e:
    print(f"❌ Error leyendo volumen: {e}")
    dbutils.notebook.exit("Volume Error")

# 2. Obtener archivos procesados (Destino)
processed_files = set()
if not FORCE_REFRESH:
    try:
        if spark.catalog.tableExists(TABLE_NAME):
            # Leemos y normalizamos lo que viene de la tabla
            df_existing = spark.table(TABLE_NAME).select("source_path").distinct().collect()
            processed_files = {normalizar_ruta(row.source_path) for row in df_existing}
            
            print(f"📊 Archivos ya en tabla ({len(processed_files)}):")
            if processed_files: print(f"   Ejemplo: {list(processed_files)[0]}")
        else:
            print("✨ La tabla no existe. Carga inicial.")
    except Exception as e:
        print(f"⚠️ Error leyendo tabla existente: {e}")

# 3. Calcular Delta (Diferencia)
# La magia de los sets: A - B
if FORCE_REFRESH:
    files_to_process = all_pdf_files
    print("🔥 FORZANDO REPROCESAMIENTO DE TODO.")
else:
    files_to_process = [f for f in all_pdf_files if f not in processed_files]

# --- REPORTE DE DEBUGGING (Esto te dirá por qué fallaba antes) ---
print("\n" + "="*40)
print(f"🔍 ANÁLISIS DE CAMBIOS")
print(f"   Total en carpeta: {len(all_pdf_files)}")
print(f"   Total en tabla:   {len(processed_files)}")
print(f"   -> NUEVOS:        {len(files_to_process)}")
print("="*40 + "\n")

if not files_to_process:
    print("✅ Todo al día. Terminando ejecución.")
    dbutils.notebook.exit("Empty")

# COMMAND ----------

# --- PROCESAMIENTO ---
new_data = []

for pdf_path in files_to_process:
    file_name = pdf_path.split("/")[-1]
    print(f"🔄 Procesando: {file_name}...")
    
    secciones = parsear_con_jerarquia(pdf_path)
    
    for sec in secciones:
        chunk_text = sec["text"]
        if len(chunk_text) < 50: continue
        
        abstract = generar_abstract(chunk_text)
        
        new_data.append({
            "id": str(uuid.uuid4()),
            "source_path": pdf_path, # Guardamos la ruta normalizada
            "section_title": sec["title"],
            "chunk_full_text": chunk_text,
            "section_abstract": abstract
        })

# Escritura
if new_data:
    print(f"\n💾 Guardando {len(new_data)} nuevos registros...")
    df_spark = spark.createDataFrame(pd.DataFrame(new_data))
    
    (df_spark.write.format("delta")
        .mode("append")
        .option("mergeSchema", "true")
        .saveAsTable(TABLE_NAME))
        
    spark.sql(f"ALTER TABLE {TABLE_NAME} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")
    print("✅ Éxito.")
else:
    print("⚠️ Archivos vacíos o ilegibles.")